In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
path = '/content/drive/My Drive/twitter_nlp'
os.chdir(path)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv(path+'/train.csv')
test = pd.read_csv(path+'/test.csv')

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel
model_name='bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.tokenize(train.iloc[0]['text'])

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train['text'], train['target'], random_state=2018, test_size=0.2)

In [ ]:
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 120
train_X_ids = pad_sequences([tokenizer(train_X.iloc[i])['input_ids'] for i in range(train_X.shape[0])],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")
val_X_ids = pad_sequences([tokenizer(val_X.iloc[i])['input_ids'] for i in range(val_X.shape[0])],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
train_masks = [[float(i != 0.0) for i in ii] for ii in train_X_ids]
val_masks = [[float(i != 0.0) for i in ii] for ii in val_X_ids]

In [ ]:
train_X_ids = torch.tensor(train_X_ids)
val_X_ids = torch.tensor(val_X_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_y = torch.tensor(train_y.values)
val_y = torch.tensor(val_y.values)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_X_ids, train_masks, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_X_ids, val_masks, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(model_name,
    num_labels=2,
    output_attentions = False,
    output_hidden_states = False)

In [ ]:
from transformers import BertForTokenClassification, AdamW
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 20
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(torch.cuda.get_device_name(0))
model = model.to('cuda:0')

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from tqdm import trange
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions, true_label  = [], []
    for step, batch in enumerate(val_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([np.argmax(i) for i in logits])
        true_label.extend(label_ids)

    eval_loss = eval_loss / len(val_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(accuracy_score(predictions, true_label)))
    print("Validation F1-Score: {}".format(f1_score(predictions, true_label)))
    torch.save(model,path+'/nlp_twitter_project')
    print('model saved!')

In [ ]:
def get_prediction(text):
  #preprocessing
  tokenized_sentence = tokenizer.encode(text)
  #predict
  with torch.no_grad():
    outputs = model(tokenized_sentence)
  logits = outputs[1].detach().cpu().numpy()
  return np.argmax(logits)

In [ ]:
test['target'] = test['text'].apply(get_prediction)

In [ ]:
test_X_ids = pad_sequences([tokenizer(test['text'].iloc[i])['input_ids'] for i in range(test.shape[0])],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")
test_masks = [[float(i != 0.0) for i in ii] for ii in test_X_ids]
test_X_ids = torch.tensor(test_X_ids)
test_masks = torch.tensor(test_masks)
test_data = TensorDataset(test_X_ids, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
predictions, true_label  = [], []
for step, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    # Telling the model not to compute or store gradients,
    # saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have not provided labels.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)
    # Move logits and labels to CPU
    logits = outputs[0].detach().cpu().numpy()
    predictions.extend([np.argmax(i) for i in logits])

In [ ]:
test['target'] = predictions

In [ ]:
test[['id','target']].to_csv('answer1_0331.csv',index=False)